In [2]:
from dotenv import load_dotenv
import os 
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [3]:
ollama_llm = Ollama(model = 'llama3')

In [4]:
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'
gpt_llm = ChatOpenAI(api_key = API_KEY,model= Model)

In [5]:
gpt_llm.invoke('what is a bot')

AIMessage(content='A bot, short for robot, is a software program that performs automated tasks on the internet. Bots can be programmed to perform a variety of functions, such as answering questions, providing information, or interacting with users in a chat or messaging platform. Bots can be simple or complex, and can be used for a wide range of purposes, including customer service, marketing, and entertainment.', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 11, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-22379d8e-e934-4422-8544-2200c1b90c5b-0')

In [6]:
parser = StrOutputParser()
gpt_chain = gpt_llm|parser
gpt_chain.invoke('what is a bot')

'A bot, short for robot, is a program or software application that performs automated tasks on the internet. Bots can be designed to perform a wide range of functions, from simple tasks like web scraping and data collection to more complex tasks like online customer service and chatbot interactions. Bots can be used for both beneficial and malicious purposes, so it is important to be cautious when interacting with them online.'

In [8]:
loader = TextLoader('data.txt',encoding = 'utf-8')
document = loader.load()

In [9]:
document

[Document(page_content='"prompt":" is there a robot - rather than using one of your existing humans.  for video creation","completion":" You can either use one of the pre-filmed templates, record yourself / upload a video, or we can hire an actor for you.  Any preference on those options?"},{"prompt":" I might try and upload my own","completion":" Okay awesome, let me know if you need any help with that"},{"prompt":" Im trying to understand how to integrate with Zapier first.  do I create a zap first ?","completion":" First you create your template video, and then you connect it to Zapier.  https://www.bhuman.ai/tutorials  Here are some step by step walkthroughs which all use Zapier"},{"prompt":" im just using the templates for now to see how this works.... ","completion":" Right, now select the app you\'d like to use and it\'ll take you to Zapier"},{"prompt":" i want to use my existing Typeform","completion":" Got it, so whenever a form is completed you generate a video, right?"},{"pr

In [11]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunks = spliter.split_documents(document)
chunks[3]

Document(page_content='first.  do I create a zap first ?","completion":" First you create your template video, and then you connect it to Zapier.  https://www.bhuman.ai/tutorials  Here are some step by step walkthroughs', metadata={'source': 'data.txt'})

In [12]:
vector_storage = FAISS.from_documents(chunks,OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

In [15]:
retriever.invoke('what is the pricing for Bhuman')

[Document(page_content='BHuman is an incredible tool for engagement, customer loyalty, differentiation, and overall success. Please click on the link below to read more about BHuman:', metadata={'source': 'data.txt'}),
 Document(page_content="can only be cloned with the paid plan. However, you'll have 7 days free trial and may cancel you subscription any time. More info about our pricing can be found here:  https://www.bhuman.ai/pricing", metadata={'source': 'data.txt'}),
 Document(page_content='our enterprise white glove services here and book a call with the team  https://www.bhuman.ai/pricing"}, {"prompt":" how do I get the lifetime deal available on appsumo?","completion":" BHuman', metadata={'source': 'data.txt'}),
 Document(page_content='with your audience at scale, whether it be leads, customers, or employees. Please click on the link below to read more about BHuman: https://docs.bhuman.ai/en/articles/6942071-what-is-bhuman"},', metadata={'source': 'data.txt'})]

In [16]:
template = ("""
You are AI-powered chatbot designed to provide 
information and assistance for customers
based on the context provided to you only. 
            
Context:{context}
Question:{question}
""")


In [17]:
prompt = PromptTemplate.from_template(template=template)
prompt.format(
    context = ' Here is a context to use',
    question = ' This is a question to answer'
)

'\nYou are AI-powered chatbot designed to provide \ninformation and assistance for customers\nbased on the context provided to you only. \n            \nContext: Here is a context to use\nQuestion: This is a question to answer\n'

In [18]:
result = RunnableParallel(context = retriever,question = RunnablePassthrough())
chain = result |prompt | gpt_llm | parser

In [19]:
chain.invoke('What is the pricing plan for Bhuman')

'The pricing plan for BHuman includes a paid plan which can only be cloned with the paid plan. However, there is a 7-day free trial available and you can cancel your subscription at any time. More information about the pricing can be found on their website at https://www.bhuman.ai/pricing. Additionally, there are different plans available such as the GROWTH plan for $39/month which is recommended for startups and includes 200 videos a month.'

In [20]:
chain.invoke('What is Bhuman')

'BHuman is a platform that allows you to personalize videos at scale by cloning the face and voice of the person in the video. It is an incredible tool for engagement, customer loyalty, differentiation, and overall success. You can read more about BHuman by clicking on the link provided: https://docs.bhuman.ai/en/articles/6942071-what-is-bhuman'

In [21]:
chain.invoke('My video is not clear')

'Based on the context provided, it seems like you may be experiencing issues with the clarity of your videos. One suggestion mentioned in the documents is to adjust the lighting for better visibility. Try using low lighting instead of bright, well-lit lighting to make it easier to see your mouth and improve the clarity of your videos. Additionally, ensure that you are following the tutorial carefully to generate better quality videos. If you need more detailed suggestions, feel free to reach out for further assistance.'